# **Импорт библиотек и загрузка данных**


In [40]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving McDonald_s_Reviews.csv to McDonald_s_Reviews (1).csv


In [ ]:
df = pd.read_csv('McDonald_s_Reviews.csv', encoding='ISO-8859-1')
df

,reviewer_id,store_name,category,store_address,latitude,longitude,rating_count,review_time,review,rating
0,1,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",3 months ago,Why does it look like someone spit on my food?...,1 star
1,2,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,It'd McDonalds. It is what it is as far as the...,4 stars
2,3,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",5 days ago,Made a mobile order got to the speaker and che...,1 star
3,4,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",a month ago,My mc. Crispy chicken sandwich was ï¿½ï¿½ï¿½ï¿...,5 stars
4,5,McDonald's,Fast food restaurant,"13749 US-183 Hwy, Austin, TX 78750, United States",30.460718,-97.792874,"1,240",2 months ago,"I repeat my order 3 times in the drive thru, a...",1 star
...,...,...,...,...,...,...,...,...,...,...
33391,33392,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",4 years ago,They treated me very badly.,1 star
33392,33393,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,The service is very good,5 stars
33393,33394,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",a year ago,To remove hunger is enough,4 stars
33394,33395,McDonald's,Fast food restaurant,"3501 Biscayne Blvd, Miami, FL 33137, United St...",25.810000,-80.189098,"2,810",5 years ago,"It's good, but lately it has become very expen...",5 stars


*pandas* - для работы с данными в виде датафреймов

*scikit-learn*

1.   Импрот *train_test_split* для разделения данных на тренировочную и тестовую выборки.
2.   Метрика *accuracy_score* - для оценки точности классификационных моделей. Она измеряет долю правильных предсказаний.
3. *LogisticRegression* -  пытается найти оптимальные веса (коэффициенты), которые минимизируют ошибку модели



# **Измерение качества константного предсказания**

In [41]:
most_frequent_rating = df['rating'].mode()[0]
constant_predictions = [most_frequent_rating] * len(df)
constant_f1 = f1_score(df['rating'], constant_predictions, average='weighted')
print(f"F1-мера константного предсказания: {constant_f1}")

F1-мера константного предсказания: 0.14475430020277247


Этот код, по сути, позволяет оценить производительность модели, которая всегда предсказывает самый часто встречающийся рейтинг. Это полезно для сравнения с другими моделями и определения, насколько хорошо другие модели справляются с задачей классификации по сравнению с простым базовым подходом.

# **Разбиение на тренировочную/тестовую выборки**

In [42]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [43]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

Разделения исходного датасета на две части. test_size=0.2 указывает, что 20% данных будут выделены для тестирования, а 80% - для обучения модели.

Гарантирует, что разделение данных будет одинаковым при каждом запуске кода. Это важно для воспроизводимости результатов и сравнения производительности моделей на одних и тех же данных.

# **Обучение бейзлайн-модели**

In [44]:
print(train_df.columns)

Index(['reviewer_id', 'store_name', 'category', 'store_address', 'latitude ',
       'longitude', 'rating_count', 'review_time', 'review', 'rating'],
      dtype='object')


In [51]:
if train_df['rating_count'].dtype == 'O':
    mean_rating_count = pd.to_numeric(train_df['rating_count'].str.replace(',', ''), errors='coerce').mean()
    train_df['rating_count'] = pd.to_numeric(train_df['rating_count'].str.replace(',', ''), errors='coerce').fillna(mean_rating_count)
    test_df['rating_count'] = pd.to_numeric(test_df['rating_count'].str.replace(',', ''), errors='coerce').fillna(mean_rating_count)

train_df.dropna(subset=['latitude ', 'longitude', 'category'], inplace=True)
test_df.dropna(subset=['latitude ', 'longitude', 'category'], inplace=True)

# Features and target variable
features = ['latitude ', 'longitude', 'rating_count', 'category']
X_train = train_df[features]
y_train = train_df['rating']
X_test = test_df[features]
y_test = test_df['rating']

# Column transformer for one-hot encoding
numeric_features = ['latitude ', 'longitude', 'rating_count']
categorical_features = ['category']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Create pipeline with preprocessor and Logistic Regression
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Make predictions
predictions = pipeline.predict(X_test)

# Evaluate the model
f1_base = f1_score(y_test, predictions, average='weighted')
print(f"F1-мера на тестовой выборке: {f1_base}")

F1-мера на тестовой выборке: 0.24674820277129864


Модель оценивается на тестовом наборе данных с использованием F1-меры, и результат выводится на экран.

# **Итог константное предскаяние и базовая модель**

In [53]:
if f1_base > constant_f1:
    print("Модель показывает лучший f1-score по сравнению с константным предсказанием.")
else:
    print("Модель не показывает лучшего f1-score по сравнению с константным предсказанием.")

Модель показывает лучший f1-score по сравнению с константным предсказанием.


# Сложная модель

In [ ]:
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import optuna

In [ ]:
def objective(trial):
    # Определение пространства поиска гиперпараметров
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.5, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 20),
        'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 20)
    }

    # Преобразование категориальных признаков в дамми-переменные
    X_train_encoded = pd.get_dummies(X_train)
    X_test_encoded = pd.get_dummies(X_test)

    # Создание и обучение модели с выбранными гиперпараметрами
    model = GradientBoostingClassifier(**params, random_state=42)
    model.fit(X_train_encoded, y_train)

    # Предсказание на тестовой выборке и вычисление accuracy
    accuracy = accuracy_score(y_test, predictions)
    f1 = f1_score(y_test, predictions, average='weighted')

    # Можно использовать взвешенную сумму метрик или другую комбинацию
    combined_score = 0.7 * accuracy + 0.3 * f1  # Например, взвешенная сумма accuracy и F1-score

    return combined_score

# Создание study объекта Optuna и запуск оптимизации
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Получение наилучших гиперпараметров
best_params = study.best_params
print("Наилучшие гиперпараметры:", best_params)


[I 2023-12-29 16:52:47,742] A new study created in memory with name: no-name-f07e43de-a5c7-45b9-bb9a-0b76652450b4
[I 2023-12-29 16:53:29,202] Trial 0 finished with value: 0.30678808054755013 and parameters: {'n_estimators': 118, 'learning_rate': 0.02374466848784745, 'max_depth': 4, 'min_samples_split': 17, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.30678808054755013.
[I 2023-12-29 16:53:35,656] Trial 1 finished with value: 0.30678808054755013 and parameters: {'n_estimators': 81, 'learning_rate': 0.06702746094397236, 'max_depth': 5, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.30678808054755013.
[I 2023-12-29 16:54:05,838] Trial 2 finished with value: 0.30678808054755013 and parameters: {'n_estimators': 294, 'learning_rate': 0.09811126027528594, 'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.30678808054755013.
[I 2023-12-29 16:54:33,573] Trial 3 finished with value: 0.30678808054755013 and parame

Наилучшие гиперпараметры: {'n_estimators': 118, 'learning_rate': 0.02374466848784745, 'max_depth': 4, 'min_samples_split': 17, 'min_samples_leaf': 1}


In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

In [54]:
# Объединение тренировочной и тестовой выборок для обработки категориальных признаков
combined_data = pd.concat([X_train, X_test], axis=0)

# Преобразование категориальных признаков в дамми-переменные для объединенных данных
combined_encoded = pd.get_dummies(combined_data)

# Разделение обратно на тренировочный и тестовый наборы
X_train_encoded = combined_encoded[:len(X_train)]
X_test_encoded = combined_encoded[len(X_train):]

# Создание и обучение модели с наилучшими гиперпараметрами на обработанных данных
best_model = GradientBoostingClassifier(**best_params, random_state=42)
best_model.fit(X_train_encoded, y_train)

# Предсказание на тестовой выборке с помощью обученной модели
best_predictions = best_model.predict(X_test_encoded)
best_f1 = f1_score(y_test, best_predictions,average='weighted')
print(f"F1 на тестовой выборке с лучшей моделью: {best_f1}")

F1 на тестовой выборке с лучшей моделью: 0.27678506710517997


Модель используется для предсказания результатов на тестовой выборке.
Рассчитываются метрики f1_score для оценки производительности модели на тестовых данных.
Результаты выводятся на экран.

In [ ]:
# Инициализация PermutationImportance и вычисление важности признаков
perm = PermutationImportance(best_model, random_state=42).fit(X_test_encoded, y_test)

# Визуализация permutation importances
eli5.show_weights(perm, feature_names=X_test_encoded.columns.tolist())

Weight,Feature
0.0444 ± 0.0039,rating_count
0.0181 ± 0.0025,latitude
0.0154 ± 0.0032,longitude
0 ± 0.0000,category_Fast food restaurant


'rating_count': Увеличение или уменьшение значения этого признака сильно влияет на предсказания модели. Это может быть ключевым показателем для прогнозирования.
'latitude': Этот признак также важен, но его влияние немного слабее 'rating_count'.
'longitude': Перестановка значений 'longitude' оказывает незначительное влияние на модель (отрицательная важность).

Эти значения важности помогают понять, какие признаки оказывают наибольшее влияние на предсказания модели, и могут помочь принять решение о том, как улучшить производительности модели.